In [10]:
import pandas as pd
import numpy as np
import datetime as dt
from fredapi import Fred
#import chart_wizard2 as cw
import matplotlib.pyplot as plt
import openpyxl
from datetime import date
from datetime import datetime, timedelta

In [16]:
def get_feddata(data_list, start_date, end_date):
    fred = Fred(api_key='5e679c4c2d59c2e2f2dd5b6807864f64')
    df_list = []
    for data_name in data_list:
        temp = fred.get_series(data_name, start = start_date, end = end_date)
        df_list.append(temp)
    data_df = pd.concat(df_list, axis=1)
    data_df.columns = data_list
    data_df = data_df.loc[start_date:end_date]
    data_df = data_df.ffill()
    
    return data_df


start_date = '2019-01-01'
end_date  = date.today()

econ_data = get_feddata(['HSN1F', 'DGS10', 'T10Y2Y','BAMLH0A0HYM2', 'CPILFESL', 'CSUSHPINSA', 'WTISPLC'], start_date, end_date)

In [24]:
T10Y_rate = econ_data.iloc[-1][['DGS10']].values[0]
T10Y_rate

4.43

In [25]:
from tqdm import tqdm
import requests

# 수집할 티커 리스트
tic_list = ['AAPL', 'MSFT', 'GOOGL']  # 예시 티커

# API 키 설정
api_key = "hT0gAk87j9xZx4PlBApvBqfVL5IahvgV"

# 결과를 저장할 리스트
metrics_list = []

for tick in tqdm(tic_list):
    try:
        # Key Metrics 엔드포인트에서 PER 수집
        key_metrics_url = f'https://financialmodelingprep.com/api/v3/key-metrics/{tick}?limit=1&apikey={api_key}'
        key_metrics_response = requests.get(key_metrics_url)
        key_metrics_data = key_metrics_response.json()

        # Ratios 엔드포인트에서 PBR, ROIC, 배당성향 수집
        ratios_url = f'https://financialmodelingprep.com/api/v3/ratios/{tick}?limit=1&apikey={api_key}'
        ratios_response = requests.get(ratios_url)
        ratios_data = ratios_response.json()

        # 데이터가 존재하는지 확인
        if key_metrics_data and ratios_data:
            # 가장 최근 데이터 추출
            key_metrics = key_metrics_data[0]
            ratios = ratios_data[0]

            # 필요한 지표 추출
            per = key_metrics.get('peRatioTTM')
            pbr = ratios.get('priceToBookRatio')
            roic = ratios.get('returnOnInvestedCapital')
            payout_ratio = ratios.get('payoutRatio')

            # 결과 저장
            metrics_list.append({
                'ticker': tick,
                'PER': per,
                'PBR': pbr,
                'ROIC': roic,
                'Dividend Payout Ratio': payout_ratio
            })
        else:
            print(f"{tick}의 데이터를 가져올 수 없습니다.")
    except Exception as e:
        print(f"{tick} 처리 중 오류 발생: {e}")

# 결과를 DataFrame으로 변환
metrics_df = pd.DataFrame(metrics_list)

# 결과 출력
print(metrics_df)

100%|██████████| 3/3 [00:04<00:00,  1.54s/it]

  ticker   PER        PBR  ROIC  Dividend Payout Ratio
0   AAPL  None  61.372438  None               0.162520
1   MSFT  None  12.641532  None               0.247016
2  GOOGL  None   7.173490  None               0.073543


In [4]:
## 금리 데이터를 가져온다
# 오늘 기준
date_info = datetime.now().strftime('%Y-%m-%d')

# FRED ticker 리스트
fred_tickers = [
    'T10Y2Y',        # 장단기 금리차
    'BAMLH0A0HYM2',  # 고수익 회사채 스프레드
    'BAMLC0A3CA',    # A급 회사채 스프레드
    'CPIAUCNS',      # 소비자물가지수
    'WTISPLC',       # 서부 텍사스 원유 가격
    'UMCSENT',       # 미시간대 소비자 심리지수
    'CSUSHPINSA',    # 주택가격지수 (Case-Shiller)
    'HSN1F',         # 신규주택 판매
    'IPMAN',         # 제조업 산업생산
    'AMTMNO'         # 제조업 신규주문
]

# 데이터를 하나씩 받아서 합치기
macro_data = pd.DataFrame()

for ticker in fred_tickers:
    df = fdr.DataReader(f'FRED:{ticker}', '2000-01-01', date_info)
    df.rename(columns={'Value': ticker}, inplace=True)
    if macro_data.empty:
        macro_data = df
    else:
        macro_data = macro_data.join(df, how='outer')

# 결과 미리 보기
print(macro_data.tail())


ValueError: Missing column provided to 'parse_dates': 'DATE'

In [6]:
df

NameError: name 'df' is not defined

In [7]:
df = fdr.DataReader('T10Y2Y', '2000-01-01', date_info)

HTTPError: 404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v8/finance/chart/T10Y2Y?period1=946652400&period2=1748876400&interval=1d&includeAdjustedClose=true